In [1]:
import os

import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# for auto-reloading
%load_ext autoreload
%autoreload 2    

In [3]:
pd.options.display.max_columns = None  # display all columns

In [4]:
# load basic settings

## load and convert gt data...

In [5]:
gt_survey = pd.read_csv(
    os.path.join(
        "..", "data_inputs", "gt_survey",
        "2022_campus_commute_survey_data_no_zip_na_05122023.csv"
    ),
    index_col = 0
)

print(gt_survey.shape)

(2472, 142)


In [6]:
# filter out nan values for zip code
gt_survey = gt_survey[gt_survey['ZIPCODE'].notna()]
gt_survey["ZIPCODE"] = gt_survey["ZIPCODE"].astype(int)

display(gt_survey.head(3))
print(gt_survey.shape)

,StartDate,EndDate,Status,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,Q66,Q73,Q73_2_TEXT,Q73_5_TEXT,Q2,Q68,Q67,Q3,Q3_6_TEXT,Q5,Q45,ZIPCODE,Q56_1,Q56_2,Q56_3,Q56_4,Q56_5,Q10,Q11,Q12_1,Q12_2,Q12_3,Q12_4,Q12_5,Q12_6,Q12_7,Q12_8,Q12_9,Q12_10,Q12_11,Q12_12,Q12_13,Q12_18,Q12_14,Q12_15,Q12_16,Q12_17,Q12_17_TEXT,Q13_1,Q13_2,Q13_3,Q13_1_TEXT,Q13_2_TEXT,Q13_3_TEXT,Q14_1,Q14_2,Q14_3,Q14_1_TEXT,Q14_2_TEXT,Q14_3_TEXT,Q15,Q16,Q16_3_TEXT,Q69,Q69_6_TEXT,Q17,Q18,Q19,Q55,Q20_1,Q20_2,Q20_3,Q20_4,Q20_5,Q20_6,Q20_7,Q20_8,Q20_9,Q20_10,Q20_11,Q20_13,Q20_14,Q20_16,Q20_17,Q20_18,Q20_19,Q20_20,Q20_21,Q20_22,Q20_22_TEXT,Q25,Q25_14_TEXT,Q64,Q26,Q26_16_TEXT,Q27,Q28,Q29,Q30_1,Q30_2,Q30_3,Q30_4,Q30_5,Q30_6,Q30_7,Q30_8,Q30_12,Q30_10,Q30_11,Q32_1,Q32_2,Q32_3,Q32_4,Q32_5,Q34_1,Q34_2,Q34_3,Q34_4,Q34_5,Q34_6,Q34_7,Q34_8,Q34_9,Q34_10,Q34_11,Q34_12,Q34_13,Q34_14,Q34_15,Q34_16,Q47,Q59,Q51,Q49,Q49_5_TEXT,Q48,Q50,Q61,Q62,Q65,Q35,Q38
4,10/31/2022 14:28,10/31/2022 14:36,IP Address,100,454,True,10/31/2022 14:36,R_3I6RkopoEqc7Hhp,email,EN,Yes,Mostly use the TransLoc app,NaN,NaN,I live off campus and work or attend class at ...,NaN,NaN,Staff member,NaN,20-39 minutes,4 to 10 miles,30344,Telecommute (work from home),Telecommute (work from home),"Drive alone (car, truck, motorcycle, moped)","Drive alone (car, truck, motorcycle, moped)","Drive alone (car, truck, motorcycle, moped)",7:30am,5:00pm,NaN,Don't like to depend on others for carpooling,NaN,Enjoy the ride/prefer to drive,NaN,NaN,NaN,NaN,Need to leave quickly in an emergency,NaN,NaN,NaN,NaN,NaN,NaN,Safety concerns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Need to save money,NaN,NaN,NaN,NaN,NaN,NaN,Nothing would alter my current driving habits,NaN,NaN,MARTA bus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program",$50 to $100,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
5,10/31/2022 14:31,10/31/2022 14:36,IP Address,100,263,True,10/31/2022 14:36,R_Olq3a5qleDd1xBv,email,EN,No,NaN,NaN,NaN,I live off campus and work or attend class at ...,NaN,NaN,Faculty member,NaN,40-59 minutes,4 to 10 miles,30310,MARTA rail,MARTA rail,MARTA rail,MARTA rail,Did not work/flexible schedue,7:30am,6:00pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West End,Midtown station,NaN,Walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","Yes, and I have used this program","No, I am not aware of this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","Yes, and I have used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","No, I am not aware of this program",NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
6,10/31/2022 14:32,10/31/2022 14:38,IP Address,100,320,True,10/31/2022 14:38,R_10xc

(2020, 142)


In [7]:
gt_survey.Q10.unique()

array(['7:30am', '8:00am', '8:30am', '11:00am', '9:00am', '10:00am',
       '7:00am', '9:30am', '8:00pm', '9:00pm', '9:30pm', '7:00pm',
       '11:30am', '6:00am', nan, '2:00pm', '6:30am', '12:00pm', '10:30am',
       '12:30pm', '12:00am - 5:30am', '3:30pm', '6:30pm', '1:00pm',
       '1:30pm', '4:00pm', '7:30pm', '5:00pm', '3:00pm', '10:30pm',
       '8:30pm', '2:30pm'], dtype=object)

In [8]:
gt_survey['Q10'] = gt_survey['Q10'].str.replace(
    "12:00am - 5:30am",
    "5:30am"
)

In [9]:
# 2023-01-01 is the pseudo date
depart_time = "2023-01-01 " + gt_survey.Q10

gt_survey['depart_time'] = pd.to_datetime(
    depart_time,
    format=r"%Y-%m-%d %I:%M%p"
)

In [11]:
def extract_newmins(df_row):
    tm = (df_row["depart_time"] - pd.to_datetime("2023-01-01")
         ).total_seconds() / 60
    return tm

gt_survey['newmin'] = gt_survey['depart_time'] - pd.to_datetime("2023-01-01")
gt_survey['newmin'] = gt_survey.apply(extract_newmins, axis=1)
gt_survey.head(3)

,StartDate,EndDate,Status,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,Q66,Q73,Q73_2_TEXT,Q73_5_TEXT,Q2,Q68,Q67,Q3,Q3_6_TEXT,Q5,Q45,ZIPCODE,Q56_1,Q56_2,Q56_3,Q56_4,Q56_5,Q10,Q11,Q12_1,Q12_2,Q12_3,Q12_4,Q12_5,Q12_6,Q12_7,Q12_8,Q12_9,Q12_10,Q12_11,Q12_12,Q12_13,Q12_18,Q12_14,Q12_15,Q12_16,Q12_17,Q12_17_TEXT,Q13_1,Q13_2,Q13_3,Q13_1_TEXT,Q13_2_TEXT,Q13_3_TEXT,Q14_1,Q14_2,Q14_3,Q14_1_TEXT,Q14_2_TEXT,Q14_3_TEXT,Q15,Q16,Q16_3_TEXT,Q69,Q69_6_TEXT,Q17,Q18,Q19,Q55,Q20_1,Q20_2,Q20_3,Q20_4,Q20_5,Q20_6,Q20_7,Q20_8,Q20_9,Q20_10,Q20_11,Q20_13,Q20_14,Q20_16,Q20_17,Q20_18,Q20_19,Q20_20,Q20_21,Q20_22,Q20_22_TEXT,Q25,Q25_14_TEXT,Q64,Q26,Q26_16_TEXT,Q27,Q28,Q29,Q30_1,Q30_2,Q30_3,Q30_4,Q30_5,Q30_6,Q30_7,Q30_8,Q30_12,Q30_10,Q30_11,Q32_1,Q32_2,Q32_3,Q32_4,Q32_5,Q34_1,Q34_2,Q34_3,Q34_4,Q34_5,Q34_6,Q34_7,Q34_8,Q34_9,Q34_10,Q34_11,Q34_12,Q34_13,Q34_14,Q34_15,Q34_16,Q47,Q59,Q51,Q49,Q49_5_TEXT,Q48,Q50,Q61,Q62,Q65,Q35,Q38,depart_time,newmin
4,10/31/2022 14:28,10/31/2022 14:36,IP Address,100,454,True,10/31/2022 14:36,R_3I6RkopoEqc7Hhp,email,EN,Yes,Mostly use the TransLoc app,NaN,NaN,I live off campus and work or attend class at ...,NaN,NaN,Staff member,NaN,20-39 minutes,4 to 10 miles,30344,Telecommute (work from home),Telecommute (work from home),"Drive alone (car, truck, motorcycle, moped)","Drive alone (car, truck, motorcycle, moped)","Drive alone (car, truck, motorcycle, moped)",7:30am,5:00pm,NaN,Don't like to depend on others for carpooling,NaN,Enjoy the ride/prefer to drive,NaN,NaN,NaN,NaN,Need to leave quickly in an emergency,NaN,NaN,NaN,NaN,NaN,NaN,Safety concerns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Need to save money,NaN,NaN,NaN,NaN,NaN,NaN,Nothing would alter my current driving habits,NaN,NaN,MARTA bus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program",$50 to $100,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,2023-01-01 07:30:00,450.0
5,10/31/2022 14:31,10/31/2022 14:36,IP Address,100,263,True,10/31/2022 14:36,R_Olq3a5qleDd1xBv,email,EN,No,NaN,NaN,NaN,I live off campus and work or attend class at ...,NaN,NaN,Faculty member,NaN,40-59 minutes,4 to 10 miles,30310,MARTA rail,MARTA rail,MARTA rail,MARTA rail,Did not work/flexible schedue,7:30am,6:00pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West End,Midtown station,NaN,Walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","Yes, and I have used this program","No, I am not aware of this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","Yes, and I have used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","No, I am not aware of this program",NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,2023-01-01 07:30:00,450.0
6,10/31/2022

In [12]:
# package for querying zip code
import pgeocode

locator = pgeocode.Nominatim("us")
location = locator.query_postal_code(30310)
print((location.latitude, location.longitude))

(33.7278, -84.4232)


In [13]:
# extract longitude/latitude for all positions!
def extract_lon_lat(df_row):
    location = locator.query_postal_code(df_row["ZIPCODE"])
    return pd.Series({
        "longitude": location.longitude,
        "latitude": location.latitude
    })

gt_survey[["ori_lon", "ori_lat"]] = gt_survey.apply(extract_lon_lat, axis=1)
display(gt_survey.head(3))

,StartDate,EndDate,Status,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,Q66,Q73,Q73_2_TEXT,Q73_5_TEXT,Q2,Q68,Q67,Q3,Q3_6_TEXT,Q5,Q45,ZIPCODE,Q56_1,Q56_2,Q56_3,Q56_4,Q56_5,Q10,Q11,Q12_1,Q12_2,Q12_3,Q12_4,Q12_5,Q12_6,Q12_7,Q12_8,Q12_9,Q12_10,Q12_11,Q12_12,Q12_13,Q12_18,Q12_14,Q12_15,Q12_16,Q12_17,Q12_17_TEXT,Q13_1,Q13_2,Q13_3,Q13_1_TEXT,Q13_2_TEXT,Q13_3_TEXT,Q14_1,Q14_2,Q14_3,Q14_1_TEXT,Q14_2_TEXT,Q14_3_TEXT,Q15,Q16,Q16_3_TEXT,Q69,Q69_6_TEXT,Q17,Q18,Q19,Q55,Q20_1,Q20_2,Q20_3,Q20_4,Q20_5,Q20_6,Q20_7,Q20_8,Q20_9,Q20_10,Q20_11,Q20_13,Q20_14,Q20_16,Q20_17,Q20_18,Q20_19,Q20_20,Q20_21,Q20_22,Q20_22_TEXT,Q25,Q25_14_TEXT,Q64,Q26,Q26_16_TEXT,Q27,Q28,Q29,Q30_1,Q30_2,Q30_3,Q30_4,Q30_5,Q30_6,Q30_7,Q30_8,Q30_12,Q30_10,Q30_11,Q32_1,Q32_2,Q32_3,Q32_4,Q32_5,Q34_1,Q34_2,Q34_3,Q34_4,Q34_5,Q34_6,Q34_7,Q34_8,Q34_9,Q34_10,Q34_11,Q34_12,Q34_13,Q34_14,Q34_15,Q34_16,Q47,Q59,Q51,Q49,Q49_5_TEXT,Q48,Q50,Q61,Q62,Q65,Q35,Q38,depart_time,newmin,ori_lon,ori_lat
4,10/31/2022 14:28,10/31/2022 14:36,IP Address,100,454,True,10/31/2022 14:36,R_3I6RkopoEqc7Hhp,email,EN,Yes,Mostly use the TransLoc app,NaN,NaN,I live off campus and work or attend class at ...,NaN,NaN,Staff member,NaN,20-39 minutes,4 to 10 miles,30344,Telecommute (work from home),Telecommute (work from home),"Drive alone (car, truck, motorcycle, moped)","Drive alone (car, truck, motorcycle, moped)","Drive alone (car, truck, motorcycle, moped)",7:30am,5:00pm,NaN,Don't like to depend on others for carpooling,NaN,Enjoy the ride/prefer to drive,NaN,NaN,NaN,NaN,Need to leave quickly in an emergency,NaN,NaN,NaN,NaN,NaN,NaN,Safety concerns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Need to save money,NaN,NaN,NaN,NaN,NaN,NaN,Nothing would alter my current driving habits,NaN,NaN,MARTA bus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program","No, I am not aware of this program",$50 to $100,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,2023-01-01 07:30:00,450.0,-84.4480,33.6919
5,10/31/2022 14:31,10/31/2022 14:36,IP Address,100,263,True,10/31/2022 14:36,R_Olq3a5qleDd1xBv,email,EN,No,NaN,NaN,NaN,I live off campus and work or attend class at ...,NaN,NaN,Faculty member,NaN,40-59 minutes,4 to 10 miles,30310,MARTA rail,MARTA rail,MARTA rail,MARTA rail,Did not work/flexible schedue,7:30am,6:00pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West End,Midtown station,NaN,Walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, but I have NOT used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","Yes, and I have used this program","No, I am not aware of this program","Yes, but I have NOT used this program","Yes, but I have NOT used this program","Yes, and I have used this program","No, I am not aware of this program","No, I am not aware of this program","Yes, and I have used this program","No, I am not aware of this program",NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,2023-

In [14]:
from pathlib import Path
Path.home()

PosixPath('/Users/geekduck')